In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import glob

sys.path.append('../')

from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
from src.utilities.config_ import morgan_train_data_path, predicted_data_path
from src.utilities.utils import save, load

In [3]:
def preprocess_df(df, outlier):
    # Group by 'Inv Date (MMM-YYYY)' and calculate the average 'Total COGS Value'
    df_grouped = df.groupby(['Inv Date (MMM-YYYY)', 'Material Code', 'Storage Location Code'], as_index=False).agg(
        {
            'Total COGS EA': 'mean',
            'Total COGS CTN': 'mean',
            'Total COGS Value': 'mean',
            'Outlier': 'first'  # Keeps the first Outlier flag (modify if needed)
        }  # Computes avg COGS and keeps first Outlier flag
    )

    # Rename columns
    df_grouped.rename(columns={
        'Total COGS EA': 'AVG Total EA',
        'Total COGS CTN': 'AVG Total CTN',
        'Total COGS Value': 'AVG Total RM'
    }, inplace=True)

    # Convert 'Inv Date (MMM-YYYY)' to datetime format
    df_grouped['Date'] = pd.to_datetime(df_grouped['Inv Date (MMM-YYYY)'], format='%b - %Y')

    # Sort DataFrame by date
    df_grouped = df_grouped.sort_values("Date")
    df_grouped.reset_index(drop=True, inplace=True)

    # Filter by outlier type
    df_grouped = df_grouped[df_grouped['Outlier'] == outlier]

    return df_grouped

def early_plot(df, column):
    # Column could be AVG Total RM, EA / CTN
    # Plot
    plt.figure(figsize=(10, 5))
    sns.lineplot(x=df["Date"], y=df[column], marker='o', linestyle='-')

    # Labels & Title
    plt.xlabel("Invoice Date")
    plt.ylabel(column)
    plt.title(f"{column} Over Time")
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.grid(True)

    # Show plot
    plt.show()

def train_predict(df, column):
    # Column could be AVG Total RM, EA / CTN
    # Sort values to ensure correct order
    df_base_sample = df.sort_values("Date").reset_index(drop=True)

    # List of seasonal values to try (prioritizing 12 and 6, then smaller values if needed)
    seasonal_options = [12, 8, 6, 4, 3, 2]

    for seasonal_period in seasonal_options:
        try:
            # Define SARIMA model
            model = SARIMAX(df_base_sample[column], 
                            order=(1,1,1),       
                            seasonal_order=(1,1,1,seasonal_period),  
                            enforce_stationarity=False,
                            enforce_invertibility=False)

            model_fit = model.fit(disp=False)
            
            # Forecast next 12 months
            forecast = model_fit.forecast(steps=24)

            # Replace negative values with 0
            forecast[forecast < 0] = 0
            
            # Explosion check: If the forecast exceeds 5 * max actual value, it's unstable
            if (forecast.abs().max() > 3 * df_base_sample[column].max()):
                print(f"⚠️ Forecast exploded with seasonal order {seasonal_period}, trying next...")
                continue
            
            print(f"Model trained successfully with seasonal order {seasonal_period}")
            return forecast  # Return forecast if successful
        
        except Exception as e:
            print(f"Model failed with seasonal order {seasonal_period}: {e}")
            continue

    print("No valid SARIMAX model could be trained. Consider removing seasonality.")
    return None  # Return None if all attempts fail

def postprocess(df_base_sample, forecast, column, plot):
    # Column could be AVG Total RM, EA / CTN
    # Get the last date in the dataset
    last_date = df_base_sample['Date'].max()

    # Generate future dates for the next 12 months, keeping the day as 1
    future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1),  # Start from next month
                                periods=24,  # Generate 12 months
                                freq='MS')   # 'MS' ensures the 1st day of each month


    if plot:
        # Plot actual data and forecast
        plt.figure(figsize=(10, 5))
        plt.plot(df_base_sample.Date, df_base_sample[column], label="Actual Data", marker='o')
        plt.plot(future_dates, forecast, label="Forecast", linestyle='dashed', marker='x', color='red')

        plt.xlabel("Date")
        plt.ylabel(column)
        plt.title(f"Corrected Forecast of {column}")
        plt.legend()
        plt.grid(True)
        plt.show()
    
    return future_dates

def all_storage_grouper(df):
    # Add this to switch Storage Location to = all
    df = df.groupby(['Inv Date (MMM-YYYY)', 'Material Code'], as_index=False).agg(
            {
                'AVG Total EA': 'mean',
                'AVG Total CTN': 'mean',
                'AVG Total RM': 'mean',
                'Outlier': 'first'  # Keeps the first Outlier flag (modify if needed)
            }  # Computes avg COGS and keeps first Outlier flag
        )   
    
    # Convert 'Inv Date (MMM-YYYY)' to datetime format
    df['Date'] = pd.to_datetime(df['Inv Date (MMM-YYYY)'], format='%b - %Y')

    # Sort DataFrame by date
    df = df.sort_values("Date")
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(morgan_train_data_path, "*.csv"))

# Read each CSV into a DataFrame and store them in a list
dataframes = [pd.read_csv(file) for file in csv_files]

# Optionally, concatenate all DataFrames into one
df = pd.concat(dataframes, ignore_index=True).reset_index(drop=True)
df

,Inv Date,Inv Date (MMM-YYYY),Material Group Code,Material Group Desc,Material Code,Material Desc,Plant Code,Storage Location Code,Storage Location Desc,Payer Customer Group,Payer Customer Group 1,Payer Customer,Total COGS EA,Total COGS CTN,Total COGS Value,Sales Department Code,Inv - Net NV1,IsFestive,Status,Outlier
0,2021-01-04,Jan - 2021,179,MORGAN,110088497,MVC-TA161DW VACUUM CLEANER(DRY/WET/BLOW),MY01,2020.0,Century ...,SPECIALITY STORES TT (SG),(),WONG ELECTRIC AND TV (M) SDN BHD (103030894),4,4.0,634.48,E6,990.00,True,Active,False
1,2021-01-04,Jan - 2021,179,MORGAN,110232844,MBH-GC522C(BK) 2 BURNER BIH GLASS,MY01,2020.0,Century ...,SPECIALITY STORES TT (SG),(),PUSAT LETRIK SAM (103020354),2,2.0,372.28,E6,622.50,True,Active,False
2,2021-01-04,Jan - 2021,179,MORGAN,110232844,MBH-GC522C(BK) 2 BURNER BIH GLASS,MY01,2020.0,Century ...,SPECIALITY STORES TT (SG),(),U & ME ELECTRICAL SDN BHD (103021142),5,5.0,930.71,E6,1600.00,True,Active,False
3,2021-01-04,Jan - 2021,179,MORGAN,110398316,MGS-7313S GAS STOVE (SS/130X130MM),MY01,2020.0,Century ...,SPECIALITY STORES TT (SG),(),U & ME ELECTRICAL SDN BHD (103021142),10,10.0,566.51,E6,1090.00,True,Active,False
4,2021-01-04,Jan - 2021,179,MORGAN,110493880,MJK-1010L JUG KETTLE (1.0L),MY01,2020.0,Century ...,SPECIALITY STORES TT (SG),(),K.H. ENTERPRISE (MIRI) SDN BHD (103020427),8,4.0,206.83,E6,447.84,True,Active,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55395,2024-12-31,Dec - 2024,179,MORGAN,110966983,MCF-1678LS CHEST FREEZER (175L),MY01,1173.0,NE Sound ...,SPECIALITY STORES (SG),(),HEELECTRICAL SALES & SERVICES (103020547),2,2.0,889.94,E6,685.00,True,Active,False
55396,2024-12-31,Dec - 2024,179,MORGAN,110989945,MCS-299 CHILLER SHOWCASE 240L FROST FREE,MY01,1173.0,NE Sound ...,SPECIALITY STORES (SG),(),THREE STAR AIR-CONDITIONER TRADING (103020298),2,2.0,1509.01,E6,2840.00,True,Active,False
55397,2024-12-31,Dec - 2024,179,MORGAN,110991041,MPG-DB899NS PAN GRILL WITH POT 2.5L,MY01,1173.0,NE Sound ...,ELECTRICAL SHOP (E2),(),INTERBASE RESOURCES SDN BHD (103022540),2,2.0,258.00,E6,362.00,True,Active,False
55398,2024-12-31,Dec - 2024,179,MORGAN,110991041,MPG-DB899NS PAN GRILL WITH POT 2.5L,MY01,1173.0,NE Sound ...,REDEMPTION HOUSE (R1),(),TRI E MARKETING SDN BHD (103021507),2,2.0,258.00,E6,362.00,True,Active,False


In [5]:
# Hyperparameters :)
COLUMN = ["AVG Total RM", "AVG Total EA", "AVG Total CTN"]
STORAGE_TYPE = ["All", "Specific"]
OUTLIER = [True, False]
LOOP_VALUE = 1

# Store a new empty list for the final result
results = []

# Iterate over all parameter combinations
for idx, (col, storage_type, outlier) in enumerate(product(COLUMN, STORAGE_TYPE, OUTLIER), start=1):
    print(f"Running {idx}/12 → COLUMN={col}, STORAGE_TYPE={storage_type}, OUTLIER={outlier}")

    # Preprocess data once per OUTLIER setting
    df_grouped = preprocess_df(df, outlier)
    # df_grouped = df_grouped[
    #     (df_grouped['Material Code'] == 110624332)
    # ].reset_index()
    
    # Apply grouping if storage_type is "All"
    if storage_type != "Specific":
        df_grouped = all_storage_grouper(df_grouped)
        material_storage_pairs = ((m, None) for m in df_grouped["Material Code"].unique())  # Use (material, None) pairs
    else:
        material_storage_pairs = product(df_grouped["Material Code"].unique(), df_grouped["Storage Location Code"].unique())

    # Iterate over material-storage pairs (up to LOOP_VALUE)
    for i, (material, storage) in enumerate(material_storage_pairs):
        if i >= LOOP_VALUE: 
            break  # Stop early if LOOP_VALUE is reached

        # Efficient DataFrame filtering using .query()
        if storage:
            df_sample = df_grouped.query("`Material Code` == @material and `Storage Location Code` == @storage")
        else:
            df_sample = df_grouped.query("`Material Code` == @material")

        # Skip if DataFrame is empty
        if df_sample.empty:
            continue

        try:
            # Train and forecast
            forecast = train_predict(df_sample, col)

            # Post-process and plot results
            future_dates = postprocess(df_sample, forecast, col, False)

            # Determine COGS Type
            cogs_type = "RM" if col == "AVG Total RM" else "EA" if col == "AVG Total EA" else "CTN"

            # Store results
            results.extend([
                {
                    "Date": future_date,
                    "Material Code": material,
                    "Storage Location Code": storage if storage else "All",
                    "COGS Type": cogs_type,
                    "COGS Value": value,
                    "Outlier": outlier
                }
                for future_date, value in zip(future_dates, forecast)
            ])

        except Exception as error:
            print(f"⚠️ Error for Material {material}: {error}")
            continue

# Convert results list into a DataFrame
df_results = pd.DataFrame(results)

# Display the first few rows
df_results.head()

Running 1/12 → COLUMN=AVG Total RM, STORAGE_TYPE=All, OUTLIER=True
Model trained successfully with seasonal order 12
Running 2/12 → COLUMN=AVG Total RM, STORAGE_TYPE=All, OUTLIER=False
Model trained successfully with seasonal order 12
Running 3/12 → COLUMN=AVG Total RM, STORAGE_TYPE=Specific, OUTLIER=True
Model trained successfully with seasonal order 12
Running 4/12 → COLUMN=AVG Total RM, STORAGE_TYPE=Specific, OUTLIER=False
Model trained successfully with seasonal order 12
Running 5/12 → COLUMN=AVG Total EA, STORAGE_TYPE=All, OUTLIER=True
Model trained successfully with seasonal order 12
Running 6/12 → COLUMN=AVG Total EA, STORAGE_TYPE=All, OUTLIER=False
Model trained successfully with seasonal order 12
Running 7/12 → COLUMN=AVG Total EA, STORAGE_TYPE=Specific, OUTLIER=True


/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/stats

Model trained successfully with seasonal order 12
Running 8/12 → COLUMN=AVG Total EA, STORAGE_TYPE=Specific, OUTLIER=False
Model trained successfully with seasonal order 12
Running 9/12 → COLUMN=AVG Total CTN, STORAGE_TYPE=All, OUTLIER=True
Model trained successfully with seasonal order 12
Running 10/12 → COLUMN=AVG Total CTN, STORAGE_TYPE=All, OUTLIER=False
Model trained successfully with seasonal order 12
Running 11/12 → COLUMN=AVG Total CTN, STORAGE_TYPE=Specific, OUTLIER=True
Model trained successfully with seasonal order 12
Running 12/12 → COLUMN=AVG Total CTN, STORAGE_TYPE=Specific, OUTLIER=False


/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/opt/anaconda3/envs/market-dash/lib/python3.9/site-packages/statsm

Model trained successfully with seasonal order 12


,Date,Material Code,Storage Location Code,COGS Type,COGS Value,Outlier
0,2021-02-01,110825702,All,RM,17978.18,True
1,2021-03-01,110825702,All,RM,17978.18,True
2,2021-04-01,110825702,All,RM,17978.18,True
3,2021-05-01,110825702,All,RM,17978.18,True
4,2021-06-01,110825702,All,RM,17978.18,True


In [7]:
df_results["Storage Location Code"] = df_results["Storage Location Code"].astype(str)
save(df_results, os.path.join(predicted_data_path, "morgan_test.feather"))

In [10]:
load(os.path.join(predicted_data_path, "morgan_predicted_full.feather"))

,Date,Material Code,Storage Location Code,COGS Type,COGS Value,Outlier
0,2021-02-01,110825702,All,RM,17978.18,True
1,2021-03-01,110825702,All,RM,17978.18,True
2,2021-04-01,110825702,All,RM,17978.18,True
3,2021-05-01,110825702,All,RM,17978.18,True
4,2021-06-01,110825702,All,RM,17978.18,True
...,...,...,...,...,...,...
99259,2026-08-01,111051610,1173.0,CTN,0.00,False
99260,2026-09-01,111051610,1173.0,CTN,0.00,False
99261,2026-10-01,111051610,1173.0,CTN,0.00,False
99262,2026-11-01,111051610,1173.0,CTN,0.00,False
